# CNN Implementation for Cat Breed Classification

This notebook implements a Convolutional Neural Network (CNN) to classify cat breeds.

**Objectives:**
1.  **Model Architecture**: Define a CNN with Batch Normalization and Dropout layers.
2.  **Kernel Size Experiment**: Compare the performance of different kernel sizes (e.g., 3x3, 5x5, 7x7).
3.  **Evaluation**: Select the best kernel size and visualize Training vs. Validation loss and accuracy.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


## 1. Data Loading

In [2]:
# Hyperparameters
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 10  # Increased slightly to see convergence differences
TARGET_SIZE = (224, 224)

# Data Directories
TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'

# Transforms
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load Datasets
train_dataset = datasets.ImageFolder(root=TRAIN_DIR, transform=transform)
test_dataset = datasets.ImageFolder(root=TEST_DIR, transform=transform)

# Data Loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f'Classes: {len(train_dataset.classes)}')
print(f'Training samples: {len(train_dataset)}')
print(f'Testing samples: {len(test_dataset)}')

Classes: 66
Training samples: 18054
Testing samples: 2257


## 2. Model Definition
Defining a CNN class that includes **Batch Normalization** and **Dropout** by default. The kernel size is parameterized.

In [3]:
class CatBreedCNN(nn.Module):
    def __init__(self, num_classes=66, kernel_size=3):
        super(CatBreedCNN, self).__init__()
        # Calculate padding to maintain spatial dimensions (same padding)
        # padding = (kernel_size - 1) / 2
        padding = kernel_size // 2
        
        # Layer 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=kernel_size, padding=padding)
        self.bn1 = nn.BatchNorm2d(32)
        
        # Layer 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=padding)
        self.bn2 = nn.BatchNorm2d(64)
        
        # Layer 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=kernel_size, padding=padding)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        
        # Calculate input size for FC layer
        # Input: 224x224
        # After Pool 1: 112x112
        # After Pool 2: 56x56
        # After Pool 3: 28x28
        self.fc_input_size = 128 * 28 * 28
        
        self.fc1 = nn.Linear(self.fc_input_size, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Block 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # Classifier
        x = x.view(-1, self.fc_input_size)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

## 3. Training Function
Includes validation tracking at each epoch.

In [4]:
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs=NUM_EPOCHS):
    model.to(device)
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    print(f"Training on {device}...")
    
    for epoch in range(num_epochs):
        # --- Training Phase ---
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
        avg_train_loss = train_loss / len(train_loader)
        avg_train_acc = 100 * train_correct / train_total
        
        # --- Validation Phase ---
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                
        avg_val_loss = val_loss / len(val_loader)
        avg_val_acc = 100 * val_correct / val_total
        
        # --- Record History ---
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(avg_train_acc)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(avg_val_acc)
        
        print(f'Epoch [{epoch+1}/{num_epochs}]: '
              f'Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.2f}% | '
              f'Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.2f}%')
              
    return history

## 4. Experiments: Kernel Sizes
Comparing Kernel Sizes: 3, 5, and 7.

In [ ]:
kernel_sizes = [3, 5, 7]
results = {}

for k in kernel_sizes:
    print(f"\n{'='*40}")
    print(f"Experiment: Kernel Size = {k}")
    print(f"{'='*40}")
    
    model = CatBreedCNN(kernel_size=k)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()
    
    history = train_and_validate(model, train_loader, test_loader, criterion, optimizer, num_epochs=NUM_EPOCHS)
    results[k] = history


Experiment: Kernel Size = 3
Training on cpu...


## 5. Results & Visualization

In [ ]:
# 1. Compare Kernel Sizes (Validation Accuracy)
plt.figure(figsize=(10, 6))
for k in kernel_sizes:
    plt.plot(results[k]['val_acc'], label=f'Kernel Size {k}')
plt.title('Validation Accuracy vs. Epochs for Different Kernel Sizes')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy (%)')
plt.legend()
plt.grid(True)
plt.show()

# Determine Best Kernel Size
best_k = max(results, key=lambda k: max(results[k]['val_acc']))
print(f"\nBest Kernel Size based on max validation accuracy: {best_k}")

In [ ]:
# 2. Detailed Plots for Best Model
best_history = results[best_k]

plt.figure(figsize=(14, 5))

# Loss Plot
plt.subplot(1, 2, 1)
plt.plot(best_history['train_loss'], label='Train Loss')
plt.plot(best_history['val_loss'], label='Validation Loss')
plt.title(f'Best Model (Kernel {best_k}): Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Accuracy Plot
plt.subplot(1, 2, 2)
plt.plot(best_history['train_acc'], label='Train Accuracy')
plt.plot(best_history['val_acc'], label='Validation Accuracy')
plt.title(f'Best Model (Kernel {best_k}): Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

plt.show()